# Introduction

context management protocol allows you to create your own context managers so you can customize the way you deal with system resources.

# The try … finally Approach
Working with files is probably the most common example of resource management in programming. In Python, you can use a try … finally statement to handle opening and closing files properly

In [1]:
# Safely open the file
file = open("hello.txt", "w")

try:
    # The try block in the above example can potentially 
    # raise exceptions, such as AttributeError or NameError. 
    file.write("Hello, World!")
except Exception as e:
    # You can handle those exceptions in an except clause
    print(f"An error occurred while writing to the file: {e}")
finally:
    # Make sure to close the file after using it
    #  the finally clause will guarantee that file is properly closed, 
    # even if an exception occurs during the call to .write() in the 
    # try clause
    file.close()

In this example, you catch any potential exceptions that can occur while writing to the file. In real-life situations, you should use a specific exception type instead of the general Exception to prevent unknown errors from passing silently.

# The with Statement Approach

The Python with statement creates a runtime context that allows you to run a group of statements under the control of a context manager. Compared to traditional **try … finally** constructs, the with statement can make your code **clearer, safer, and reusable.** 
To write a with statement, you need to use the following general syntax:
```
with expression as target_var:
    do_something(target_var)
```
The context manager object results from evaluating the expression after with. In other words, expression must return an object that implements the **context management protocol**. This protocol consists of two special methods:

1. **.\_\_enter\_\_()** is called by the with statement to enter the runtime context. typically provides the setup code. 
2. **.\_\_exit\_\_()** is called when the execution leaves the with code block. This method typically provides the teardown logic or cleanup code, such as calling .close() on an open file object

The as specifier is optional. If you provide a target_var with as, then the return value of calling **.\_\_enter\_\_()** on the context manager object is bound to that variable.

**Note:** Some context managers return None from .__enter__() because they have no useful object to give back to the caller. In these cases, specifying a target_var makes no sense.


Here’s how the with statement proceeds when Python runs into it:
1. Call expression to obtain a context manager.
2. Store the context manager’s **.\_\_enter\_\_()** and **.\_\_exit\_\_()** methods for later use.
3. Call **.\_\_enter\_\_()** on the context manager and bind its return value to target_var if provided.
4. Execute the with code block.
5. Call **.\_\_exit\_\_()** on the context manager when the with code block finishes.

In [3]:
"""
When you run this with statement, open() returns an io.TextIOBase object. 
This object is also a context manager, so the with statement calls 
.__enter__() and assigns its return value to file. Then you can manipulate 
the file inside the with code block. When the block ends, .__exit__() 
automatically gets called and closes the file for you, even if an exception
is raised inside the with block.
"""
with open("hello.txt", mode="w") as file:
    file.write("Hello, World!")

In Python 3.1 and later, the with statement supports multiple context managers. You can supply any number of context managers separated by commas:
```
with A() as a, B() as b:
    pass
```

This works like nested with statements but without nesting. This might be useful when you need to open two files at a time, the first for reading and the second for writing:

In [5]:
"""
In this example, you can add code for reading and transforming the content of input.txt. 
Then you write the final result to output.txt in the same code block.
"""
with open("input.txt") as in_file, open("output.txt", "w") as out_file:
    # Read content from input.txt
    # Transform the content
    # Write the transformed content to output.txt
    pass

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

# Case Study of with Statement

## Working With Files
Opening files using the with statement is generally recommended because it ensures that open file descriptors are automatically closed after the flow of execution leaves the with code block.

In [6]:
with open("hello.txt", mode="w") as file:
    file.write("Hello, World!")

Finally, whenever you load an external file, your program should check for possible issues, such as a missing file, writing and reading access, and so on. Here’s a general pattern that you should consider using when you’re working with files:

In [7]:
import pathlib
import logging

file_path = pathlib.Path("hello.txt")

try:
    with file_path.open(mode="w") as file:
        file.write("Hello, World!")
except OSError as error:
    logging.error("Writing to file %s failed due to: %s", file_path, error)

## Traversing Directories
The os module provides a function called scandir(), which returns an iterator over os.DirEntry objects corresponding to the entries in a given directory. This function is specially designed to provide optimal performance when you’re traversing a directory structure.

In [8]:
import os
with os.scandir(".") as entries:
    for entry in entries:
        print(entry.name, "->", entry.stat().st_size, "bytes")

.ipynb_checkpoints -> 4096 bytes
Bit_Operation.ipynb -> 14657 bytes
Common Data Struture.ipynb -> 40540 bytes
data -> 4096 bytes
Data Classes.ipynb -> 18070 bytes
decorator.ipynb -> 48244 bytes
Gnerator.ipynb -> 1665 bytes
hello.txt -> 13 bytes
image -> 4096 bytes
Iteration, Iterables, Iterators, Looping and Comprehension.ipynb -> 41678 bytes
Mutable vs Immutable Objects.ipynb -> 25664 bytes
Number System.ipynb -> 7692 bytes
resource management.ipynb -> 12238 bytes
Shallow and Deep Copy.ipynb -> 4746 bytes
subarray.ipynb -> 7219 bytes
Timer.ipynb -> 5392 bytes


## Performing High-Precision Calculations

Unlike built-in floating-point numbers, the decimal module provides a way to adjust the precision to use in a given calculation that involves Decimal numbers. The precision defaults to 28 places, but you can change it to meet your problem requirements. A quick way to perform calculations with a custom precision is using localcontext() from decimal:

In [11]:
"""
Here, localcontext() provides a context manager that creates a local decimal 
context and allows you to perform calculations using a custom precision. 
In the with code block, you need to set .prec to the new precision you want to use, 
which is 42 places in the example above. When the with code block finishes, 
the precision is reset back to its default value, 28 places.
"""
from decimal import Decimal, localcontext
with localcontext() as ctx:
    ctx.prec = 42
    r1 = Decimal("1") / Decimal("42")
    print(r1)

r2 = Decimal("1") / Decimal("42")
print(r2)

0.0238095238095238095238095238095238095238095
0.02380952380952380952380952381


## Handling Locks in Multithreaded Programs
Another good example of using the with statement effectively in the Python standard library is **threading.Lock**. This class provides a primitive lock to prevent multiple threads from modifying a shared resource at the same time in a multithreaded application.

In [14]:
import threading

balance_lock = threading.Lock()

# Use the try ... finally pattern
balance_lock.acquire()
try:
    # Update the account balance here ...
    pass
finally:
    balance_lock.release()

# Use the with pattern
with balance_lock:
    """
    The with statement in the second example automatically acquires and releases 
    a lock when the flow of execution enters and leaves the statement.
    """
    # Update the account balance here ...
    pass

## Testing for Exceptions With pytest

several third-party libraries include objects that support the context management protocol. For example. 

Say you’re testing your code with pytest. Some of your functions and code blocks raise exceptions under certain situations, and you want to test those cases. To do that, you can use pytest.raises(). This function allows you to assert that a code block or a function call raises a given exception.

In [17]:
1 / 0

ZeroDivisionError: division by zero

In [18]:
import pytes
with pytest.raises(ZeroDivisionError):
    1 / 0

ModuleNotFoundError: No module named 'pytes'

In [19]:
favorites = {"fruit": "apple", "pet": "dog"}

In [20]:
favorites["car"]

KeyError: 'car'

In [21]:
with pytest.raises(KeyError):
    favorites["car"]

# Using the async with Statement

The with statement also has an asynchronous version, async with. You can use it to write context managers that depend on asynchronous code. It’s quite common to see async with in that kind of code, as many IO operations involve setup and teardown phases.

In [22]:
# site_checker_v0.py

import aiohttp
import asyncio

async def check(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print(f"{url}: status -> {response.status}")
            html = await response.text()
            print(f"{url}: type -> {html[:17].strip()}")

async def main():
    await asyncio.gather(
        check("https://realpython.com"),
        check("https://pycoders.com"),
    )

asyncio.run(main())

ModuleNotFoundError: No module named 'aiohttp'

Comparing with "with" statement, "async with" statement:
1. requires an asynchronous context manager that is able to suspend execution in its enter and exit method
2. .\_\_aenter\_\_() will replace .\_\_enter\_\_() in a regular context manager.
3. .\_\_aexit\_\_() will replace .\_\_exit\_\_() in a regular context manager.
4. The async with ctx_mgr construct implicitly uses await ctx_mgr.\_\_aenter\_\_()
5. await ctx_mgr.\_\_aexit\_\_() when exiting it. 

# Creating Custom Context Managers

You can provide the same functionality by implementing both:
1. Define .\_\_enter\_\_() and the .\_\_exit\_\_() special methods in your **class-based** context managers.
2. create custom **function-based** context managers using the [contextlib.contextmanager](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager) decorator from the standard library and an appropriately coded [generator](https://realpython.com/introduction-to-python-generators/) function.

In general, context managers and the with statement aren’t limited to resource management. They allow you to provide and reuse common setup and teardown code. In other words, with context managers, you can perform any pair of operations that needs to be done before and after another operation or procedure, such as:
- Open and close
- Lock and release
- Change and reset
- Create and delete
- Enter and exit
- Start and stop
- Setup and teardown
You can provide code to safely manage any of these pairs of operations in a context manager. Then you can reuse that context manager in with statements throughout your code. This prevents errors and reduces repetitive boilerplate code. It also makes your APIs safer, cleaner, and more user-friendly.

## Coding Class-Based Context Managers

In [25]:
class HelloContextManager:
    def __enter__(self):
        print("Entering the context...")
        return "Hello, World!"

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("Leaving the context...")
        print(exc_type, exc_value, exc_tb, sep="\n")

In [26]:
with HelloContextManager() as hello:
    print(hello)

Entering the context...
Hello, World!
Leaving the context...
None
None
None


Now that you know how to implement the context management protocol, you can get a sense of what this would look like by coding a practical example. Here’s how you can take advantage of open() to create a context manager that opens files for writing:

In [27]:
# Opening Files for Writing: First Version
# writable.py

class WritableFile:
    def __init__(self, file_path):
        self.file_path = file_path

    def __enter__(self):
        self.file_obj = open(self.file_path, mode="w")
        return self.file_obj

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file_obj:
            self.file_obj.close()
with WritableFile("hello.txt") as file:
    file.write("Hello, World!")

A subtle detail to consider when you’re writing your own context managers is that sometimes you don’t have a useful object to return from .__enter__() and therefore to assign to the with target variable. In those cases, you can return None explicitly or you can just rely on Python’s implicit return value, which is None as well.

In [28]:
# Redirecting the Standard Output
import sys

class RedirectedStdout:
    def __init__(self, new_output):
        self.new_output = new_output

    def __enter__(self):
        self.saved_output = sys.stdout
        sys.stdout = self.new_output

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self.saved_output

with open("hello.txt", "w") as file:
    with RedirectedStdout(file):
        print("Hello, World!")
    print("Back to the standard output...")

Back to the standard output...


Just like every other class, a context manager can encapsulate some internal state. The following example shows how to create a stateful context manager to measure the execution time of a given code block or function:

In [29]:
# Measuring Execution Time
# timing.py

from time import perf_counter

class Timer:
    def __enter__(self):
        self.start = perf_counter()
        self.end = 0.0
        return lambda: self.end - self.start

    def __exit__(self, *args):
        self.end = perf_counter()

In [30]:
from time import sleep
with Timer() as timer:
    # Time-consuming code goes here...
    sleep(0.5)

## Creating Function-Based Context Managers
Python’s generator functions and the contextlib.contextmanager decorator provide an alternative and convenient way to implement the context management protocol. If you decorate an appropriately coded generator function with @contextmanager, then you get a function-based context manager that automatically provides both required methods, .\_\_enter\_\_() and .\_\_exit\_\_(). This can make your life more pleasant by saving you some boilerplate code.

The general pattern to create a context manager using @contextmanager along with a generator function goes like this:

In [31]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
    # Before the yield statement, you have the setup section.
    # There, you can place the code that acquires the managed resources.
    # Everything before the yield runs when the flow of execution enters the context.
    print("Entering the context...")  
    
    yield "Hello, world fro yield"
    
    # After the yield statement, you have the teardown section, in which you can release 
    # the resources and do the cleanup. The code after yield runs at the end of the with 
    # block. The yield statement itself provides the object that will be assigned to the 
    # with target variable.
    print("Leaving the context...")

    
with hello_context_manager() as hello:
    print(hello)

Entering the context...
Hello, world fro yield
Leaving the context...


In [32]:
# Opening Files for Writing: Second Version
from contextlib import contextmanager

@contextmanager
def writable_file(file_path):
    file = open(file_path, mode="w")
    try:
        yield file
    finally:
        file.close()

with writable_file("hello.txt") as file:
    file.write("Hello, World!")

In [34]:
# Mocking the Time
from contextlib import contextmanager
from time import time

@contextmanager
def mock_time():
    global time
    saved_time = time
    time = lambda: 42
    yield
    time = save_time
with mock_time():
    print(f"Mocked time: {time()}")

Mocked time: 42


NameError: name 'save_time' is not defined

# Reference 
1. [Context Managers and Python's with Statement](https://realpython.com/python-with-statement/)
2. [The “with” Statement](https://peps.python.org/pep-0343/)